In [65]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

import mysql.connector
from mysql.connector import Error
import psycopg2
import psycopg2.extras
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from timeit import default_timer as timer
import geopandas as gpd
from shapely.geometry import shape
from shapely.geometry import Point
from descartes import PolygonPatch
from geopandas.tools import sjoin
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import itertools
import haversine
import getpass
import os.path
import fiona
import json
import glob
import ast
import csv
import re
import itertools
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd

### Start the gazetteer DB server

Make sure you change your credentials. This is to connect to the DB locally in MySQL:

In [66]:
# read Credentials from the credentials.json file
credentials_config = dict()
with open('../credentials.json') as f:
    credentials_config = json.load(f)

In [67]:
credentials_config['lwmrelationaldb']['password'] = getpass.getpass(prompt='Enter your password: ')

Enter your password: ········


In [68]:
# Construct connection string
psql_conn_string =\
    "host={0} user={1} dbname={2} password={3} sslmode={4} sslrootcert={5}".format(
                credentials_config['lwmrelationaldb']['host'], 
                credentials_config['lwmrelationaldb']['user'], 
                'gazetteer', 
                credentials_config['lwmrelationaldb']['password'], 
                credentials_config['lwmrelationaldb']['sslmode'], 
                credentials_config['lwmrelationaldb']['sslrootcert'])

In [69]:
gazDB = psycopg2.connect(psql_conn_string) 
print("Connection established!")

cursorGaz = gazDB.cursor(cursor_factory=psycopg2.extras.DictCursor)

Connection established!


### Read GB1900

In [70]:
gb1900df = pd.DataFrame()
with open("/Users/mcollardanuy/Documents/workspace/toponymResolution/GIR2019/gb1900_analysis/gb1900_gazetteer_complete_july_2018.csv", encoding='UTF-16') as f:
    gb1900df = pd.read_csv(f)

/Users/mcollardanuy/anaconda3/envs/GIS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
gb1900df.head()

,pin_id,final_text,nation,local_authority,parish,osgb_east,osgb_north,latitude,longitude,notes
0,52b34d8b695fe90005004e1e,F. P.,Wales,Powys,Llansilin,320836.712742,327820.182715,52.842050,-3.176744,NaN
1,5800a6b92c66dcab3d061796,Parly. & Munl Boro. By.,England,City of London,NaN,531794.825962,180705.741898,51.509918,-0.102246,NaN
2,5800a6782c66dcab3d061786,S. Ps.,England,City of London,NaN,531736.217116,180725.027730,51.510105,-0.103083,NaN
3,57f684f42c66dcab3d01c0dd,Southwark Bridge Stairs,England,City of London,NaN,532199.584123,180696.934434,51.509744,-0.096420,NaN
4,57f685002c66dcab3d01c0e9,St. Paul's Pier,England,City of London,NaN,531987.486097,180745.664556,51.510232,-0.099456,NaN


### Create an approximately British Wiki Gazetteer

In [72]:
start_time = timer()

In [76]:
def find_british_locations(cursorGaz, timer):
    print('Start locations SQL query: {} seconds'.format(timer() - start_time))

    cursorGaz.execute("""
            SELECT * FROM location
            WHERE lat > 50.0
            AND lat < 62.0
            AND lon > -14.0
            AND lon < 3.0
        """)
    results = cursorGaz.fetchall()
    
    main_id = []
    wiki_title = []
    wiki_lat = []
    wiki_lon = []
    page_len = []
    type_loc = []
    population = []
    for r in results:
        main_id.append(r['id'])
        wiki_title.append(r['wiki_title'])
        page_len.append(r['page_len'])
        wiki_lat.append(r['lat'])
        wiki_lon.append(r['lon'])
        type_loc.append(r['type'])
        population.append(r['population'])

    df = pd.DataFrame(
        {'main_id' : main_id,
         'wiki_title': wiki_title,
         'wiki_lat': wiki_lat,
         'wiki_lon': wiki_lon,
         'page_len': page_len,
         'type_loc': type_loc,
         'population': population
        })
    return df

def gaz_to_geodataframe(df):
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.wiki_lon, df.wiki_lat))

    poly  = gpd.GeoDataFrame.from_file('gb1900_analysis/shapefiles/GBR_adm/GBR_adm0.shp')
    point = gdf

    pointInPolys = sjoin(point, poly, how='left')
    grouped = pointInPolys.groupby('index_right')
    
    britdf = df.iloc[grouped.groups[0]]
    britdf.reset_index(drop=True)
    britdf.to_pickle("brit_wikigazetteer.pkl")
    
    return britdf

def find_british_altnames(cursorGaz, timer):
    print('Start altnames SQL query: {} seconds'.format(timer() - start_time))

    cursorGaz.execute("""
            SELECT altname.* FROM altname
            JOIN location ON location.id=altname.main_id
            WHERE lat > 50.0
            AND lat < 62.0
            AND lon > -14.0
            AND lon < 3.0
        """)
    results = cursorGaz.fetchall()
    
    dAltnames = dict()
    
    for r in results:
        if len(r['altname']) < 50:
            dAltnames[r['id']] = (r['altname'], r['source'], r['main_id'])

    return dAltnames

#### Locations dataframe

In [77]:
# locdf = find_british_locations(cursorGaz, timer)
# britdf = gaz_to_geodataframe(locdf)
# britdf.head()
britdf = pd.read_pickle("GIR2019/gb1900_analysis/brit_wikigazetteer.pkl")
britdf.head()

,wiki_id,wiki_title,wiki_lat,wiki_lon,geometry
0,20,Aberdeenshire,57.1510,-2.12300,POINT (-2.123 57.151)
1,21,A._A._Milne,51.0900,0.10700,POINT (0.107 51.09)
4,48,Abbotsford_House,55.5997,-2.78194,POINT (-2.78194 55.5997)
5,59,Aberdeen,57.1500,-2.11000,POINT (-2.11 57.15)
6,99,"Angus,_Scotland",56.6667,-2.91667,POINT (-2.91667 56.6667)


#### Altnames dataframe

In [78]:
dAltnames = find_british_altnames(cursorGaz, timer)

alt_id = []
main_id = []
altname = []
source = []
for r in dAltnames:
    main_id.append(dAltnames[r][2])
    alt_id.append(r)
    altname.append(dAltnames[r][0])
    source.append(dAltnames[r][1])

altdf = pd.DataFrame(
    {'alt_id' : alt_id,
     'main_id': main_id,
     'altname': altname,
     'source': source
    })

Start altnames SQL query: 48.58997783399991 seconds


In [80]:
altdf.shape

(224386, 4)

In [81]:
altdf = altdf.groupby("main_id")['altname'].apply(', '.join)
altdf.head()

main_id
20    Aberdeenshire, Aberdeen County, Swydd Aberdeen, Aberdeen, Aiberdeenshire, Contae Obar Deathain, Siorrachd Obar Dheathain, Aberdonensis, Coontae Aberdon
21    A. A. Milne                                                                                                                                            
23    Azincourt, Asincurtis                                                                                                                                  
25    Achill Island, Acaill, Achill, Wyspa Achill, Achill - Acaill, Curraun                                                                                  
48    Abbotsford House, Abbotsford, Clartyhole                                                                                                               
Name: altname, dtype: object

#### Joint locations and altnames dataframe

In [86]:
britdf = pd.merge(locdf, altdf, how='left', on='main_id')
britdf = britdf[britdf.altname.notnull()]
# britdf.head()

# Example of multiple altnames:
britdf[britdf['main_id'] == 20]

,main_id,wiki_title,wiki_lat,wiki_lon,page_len,type_loc,population,geometry,altname
64008,20,Aberdeenshire,57.151,-2.123,30952,adm2nd,NaN,POINT (-2.123 57.151),"Aberdeenshire, Aberdeen County, Swydd Aberdeen, Aberdeen, Aiberdeenshire, Contae Obar Deathain, Siorrachd Obar Dheathain, Aberdonensis, Coontae Aberdon"


In [88]:
altdf.shape

(206134,)

### Exploratory first step

In [42]:
l_altname = []
l_wiki_title = []
l_gb1900text = []
l_gb1900lat = []
l_gb1900lon = []
l_wikilat = []
l_wikilon = []
for index, row in britdf.iloc[180100:180200].iterrows():
    for altnwiki in row['altname'].split(","):
        altnwiki = altnwiki.strip()
        matches = gb1900df[(gb1900df['final_text'].str.contains(row['altname'], case=False)) & (gb1900df['latitude'] >= row['wiki_lat'] - 0.0) & (gb1900df['latitude'] <= row['wiki_lat'] + 0.1) & (gb1900df['longitude'] >= row['wiki_lon'] - 0.1) & (gb1900df['longitude'] <= row['wiki_lon'] + 0.1)]
        for imatch, irow in matches.iterrows():
            l_altname.append(altnwiki)
            l_wiki_title.append(row['wiki_title'])
            l_gb1900text.append(irow['final_text'])
            l_gb1900lat.append(irow['latitude'])
            l_gb1900lon.append(irow['longitude'])
            l_wikilat.append(row['wiki_lat'])
            l_wikilon.append(row['wiki_lon'])
        if matches.empty:
            l_altname.append(altnwiki)
            l_wiki_title.append(row['wiki_title'])
            l_gb1900text.append('')
            l_gb1900lat.append('')
            l_gb1900lon.append('')
            l_wikilat.append(row['wiki_lat'])
            l_wikilon.append(row['wiki_lon'])

matchdf = pd.DataFrame(
        {'altname' : l_altname,
         'wiki_title': l_wiki_title,
         'gb1900text': l_gb1900text,
         'gb1900lat': l_gb1900lat,
         'gb1900lon': l_gb1900lon,
         'wiki_lat': l_wikilat,
         'wiki_lon': l_wikilon
        })

In [44]:
matchdf.to_pickle("match09.pkl")

In [45]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
matchdf = pd.read_pickle("match09.pkl")
matchdf

,altname,wiki_title,gb1900text,gb1900lat,gb1900lon,wiki_lat,wiki_lon
0,Copinsay Lighthouse,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",,,,58.8965,-2.67199
1,Deerness,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",ST. ANDREWS AND DEERNESS,58.9473,-2.87704,58.9329,-2.79425
2,Deerness,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",Deerness,58.9476,-2.75038,58.9329,-2.79425
3,Toab,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",,,,58.9175,-2.80488
4,Tankerness Meal Mill,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",,,,58.9614,-2.84581
5,Tankerness Fishing Station,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",,,,58.9621,-2.83201
6,Tankerness,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",,,,58.9692,-2.79546
7,,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",,,,58.9692,-2.79546
8,Canniemyre,"List_of_listed_buildings_in_St_Andrews_And_Deerness,_Orkney",,,,58.9198,-2.85532
9,Gate Piers Balfour Castle,"List_of_listed_buildings_in_Shapinsay,_Orkney",,,,59.0311,-2.91036


### Close DB connection

In [ ]:
# PostreSQL:
if(gazDB):
    cursorGaz.close()
    gazDB.close()